# Deep Agents Integration Demo

Демонстрация полной интеграции Deep Agents middleware в StatefulAgent.

## Компоненты:

1. **TodoListMiddleware** - управление task list через `write_todos` tool
2. **FilesystemMiddleware** - context eviction для больших результатов
3. **SubAgentMiddleware** - делегирование задач через `task` tool
4. **SummarizationMiddleware** - LLM-based сжатие контекста при превышении токенов
5. **File Tools** - edit_file, grep для работы с файлами

In [ ]:
# Setup imports
import sys
import asyncio
from pathlib import Path

# Add SDK to path
sys.path.insert(0, str(Path.cwd()))

from agentarea_agents_sdk.agents.stateful_agent import StatefulAgent
from agentarea_agents_sdk.tools.file_toolset import FileToolset
from agentarea_agents_sdk.middleware.summarization import count_messages_tokens

## 1. Базовая инициализация StatefulAgent с middleware

По умолчанию StatefulAgent включает все Deep Agents middleware.

In [ ]:
# Создаём агента с всеми default middleware
agent = StatefulAgent(
    name="DeepAgent",
    instruction="You are a highly capable AI agent with planning and delegation abilities.",
    model_provider="ollama_chat",
    model_name="qwen2.5:3b",
    endpoint_url="http://localhost:11434",
    temperature=0.7,
    max_tokens=2000,
    max_iterations=15,
    enable_default_middleware=True,  # Включаем все middleware
    enable_todolist=True,
    enable_filesystem=True,
    enable_subagents=True,
    enable_summarization=True,
    max_tokens_before_summary=50_000,
    messages_to_keep=6,
)

print(f"✓ Агент '{agent.name}' создан")
print(f"  Middleware: {[type(m).__name__ for m in agent.middlewares.middlewares]}")
print(f"  Tools: {[tool.name for tool in agent.tool_executor.registry.list_tools()]}")

## 2. TodoListMiddleware - планирование сложных задач

Агент использует `write_todos` tool для декомпозиции сложных задач.

In [ ]:
# Задача, требующая планирования
task = """
Analyze the StatefulAgent implementation and create a report with:
1. List all middleware types used
2. Explain how each middleware integrates
3. Identify the order of middleware execution
4. Suggest one improvement
"""

print("🎯 Задача:", task.strip())
print("\n" + "="*80 + "\n")

# Запускаем агента
async def run_planning_demo():
    result = ""
    async for chunk in agent.run_stream(task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_planning_demo()
print("\n" + "="*80)
print("\n✓ Задача выполнена")

## 3. FilesystemMiddleware - работа с файлами

Демонстрация file tools: save_file, read_file, edit_file, grep.

In [ ]:
import shutil

# Создаём тестовый проект
test_dir = Path("/tmp/demo_deep_agents")
if test_dir.exists():
    shutil.rmtree(test_dir)
test_dir.mkdir(parents=True, exist_ok=True)

# Агент с file tools
file_toolset = FileToolset(
    base_dir=test_dir,
    save_files=True,
    read_files=True,
    edit_files=True,
    grep_files=True,
)

agent_with_files = StatefulAgent(
    name="FileAgent",
    instruction="You are an agent that works with files.",
    model_provider="ollama_chat",
    model_name="qwen2.5:3b",
    endpoint_url="http://localhost:11434",
    tools=[file_toolset],
    enable_default_middleware=True,
)

print(f"✓ Создан тестовый проект: {test_dir}")
print(f"✓ Агент с file tools: {[tool.name for tool in agent_with_files.tool_executor.registry.list_tools()]}")

In [ ]:
# Задача с файлами
file_task = """
1. Create a Python file 'config.py' with:
   - DATABASE_URL = "sqlite:///app.db"
   - API_KEY = "secret123"
   - DEBUG = True

2. Read the file back to verify

3. Replace 'secret123' with 'production_key_456'

4. Search for 'production' in all .py files
"""

print("🎯 Задача:", file_task.strip())
print("\n" + "="*80 + "\n")

async def run_file_demo():
    result = ""
    async for chunk in agent_with_files.run_stream(file_task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_file_demo()
print("\n" + "="*80)
print("\n✓ Файловые операции выполнены")

# Проверяем результат
config_path = test_dir / "config.py"
if config_path.exists():
    print("\n📄 Содержимое config.py:")
    print(config_path.read_text())

## 4. SubAgentMiddleware - делегирование через task tool

Агент может делегировать подзадачи изолированным subagent'ам.

In [ ]:
# Определяем custom subagents
custom_subagents = [
    {
        "name": "analyzer",
        "description": "Specialized agent for code analysis and review",
        "system_prompt": "You are a code analysis expert. Analyze code thoroughly and provide detailed insights.",
    },
    {
        "name": "writer",
        "description": "Specialized agent for documentation and report writing",
        "system_prompt": "You are a technical writer. Create clear, well-structured documentation.",
    },
]

agent_with_subagents = StatefulAgent(
    name="OrchestratorAgent",
    instruction="You are an orchestrator that delegates tasks to specialized subagents.",
    model_provider="ollama_chat",
    model_name="qwen2.5:3b",
    endpoint_url="http://localhost:11434",
    temperature=0.7,
    max_iterations=20,
    enable_default_middleware=True,
    enable_subagents=True,
    subagents=custom_subagents,
)

print("✓ Orchestrator Agent создан")
print(f"  Available subagents: general-purpose, analyzer, writer")
print(f"  Tools: {[tool.name for tool in agent_with_subagents.tool_executor.registry.list_tools()]}")

In [ ]:
# Задача, требующая делегирования
delegation_task = """
I need you to create a comprehensive analysis report:

1. Use the 'analyzer' subagent to analyze the StatefulAgent middleware architecture:
   - What are the key components?
   - How do they interact?
   - What are potential bottlenecks?

2. Use the 'writer' subagent to create a summary document:
   - Title: "Deep Agents Middleware Architecture"
   - Include the analysis findings
   - Add a conclusion section

3. Synthesize both results into a final executive summary.
"""

print("🎯 Задача с делегированием:", delegation_task.strip())
print("\n" + "="*80 + "\n")

async def run_delegation_demo():
    result = ""
    async for chunk in agent_with_subagents.run_stream(delegation_task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_delegation_demo()
print("\n" + "="*80)
print("\n✓ Делегирование завершено")

## 5. SummarizationMiddleware - управление контекстом

При превышении лимита токенов middleware автоматически сжимает историю.

In [ ]:
# Агент с низким порогом для демонстрации
agent_with_summary = StatefulAgent(
    name="SummaryAgent",
    instruction="You are a helpful assistant.",
    model_provider="ollama_chat",
    model_name="qwen2.5:3b",
    endpoint_url="http://localhost:11434",
    enable_default_middleware=True,
    enable_summarization=True,
    max_tokens_before_summary=500,  # Низкий порог для демонстрации
    messages_to_keep=3,
)

print("✓ Агент с summarization создан")
print(f"  Token threshold: 500")
print(f"  Messages to keep: 3")

In [ ]:
# Симулируем длинный разговор
conversations = [
    "Tell me about Python programming language.",
    "What are the key features of Python?",
    "Explain Python's garbage collection.",
    "How does Python's GIL work?",
    "What are Python decorators?",
]

async def run_summary_demo():
    for i, question in enumerate(conversations, 1):
        print(f"\n{'='*80}")
        print(f"Question {i}: {question}")
        print(f"{'='*80}\n")
        
        async for chunk in agent_with_summary.run_stream(question):
            print(chunk, end="", flush=True)
        
        # Проверяем количество токенов
        messages = agent_with_summary.state.get("messages", [])
        token_count = count_messages_tokens(messages)
        print(f"\n\n📊 Current token count: {token_count}")
        print(f"📝 Messages in history: {len(messages)}")
        
        # Проверяем, был ли summary
        has_summary = any("summary" in msg.get("content", "").lower() for msg in messages if msg.get("role") == "system")
        if has_summary:
            print("🔄 Summarization triggered!")

await run_summary_demo()
print("\n" + "="*80)
print("\n✓ Summarization demo завершено")

## 6. FilesystemMiddleware - Context Eviction

Большие результаты инструментов автоматически сохраняются в virtual FS.

In [ ]:
from agentarea_agents_sdk.middleware.filesystem import FilesystemMiddleware

# Тестируем context eviction напрямую
middleware = FilesystemMiddleware(eviction_threshold=100)
state = {"files": {}}

# Симулируем большой tool result
tool_call = {
    "id": "call_12345",
    "function": {"name": "search_codebase"},
}

large_result = "\n".join([f"Match {i}: Some code snippet here..." for i in range(50)])

print(f"📊 Original result size: {len(large_result)} characters")
print(f"🎯 Eviction threshold: 100 characters")
print()

# Применяем middleware
result, state_update = await middleware.after_tool_call(tool_call, large_result, state)

print("📦 Result after middleware:")
print(result)
print()

if state_update and "files" in state_update:
    print("✓ Result evicted to virtual filesystem:")
    for path, content in state_update["files"].items():
        print(f"  {path}: {len(content)} characters")

## 7. Комплексная демонстрация всех компонентов

Финальный пример, объединяющий все middleware.

In [ ]:
# Создаём полноценного Deep Agent
deep_agent = StatefulAgent(
    name="FullDeepAgent",
    instruction="""You are a highly capable AI agent with:
    - Planning abilities (write_todos)
    - Task delegation (task tool for subagents)
    - File operations (save, read, edit, grep)
    - Context management (automatic summarization)
    
    Use all your capabilities effectively to complete complex tasks.
    """,
    model_provider="ollama_chat",
    model_name="qwen2.5:3b",
    endpoint_url="http://localhost:11434",
    temperature=0.7,
    max_tokens=2000,
    max_iterations=25,
    tools=[file_toolset],
    enable_default_middleware=True,
    enable_todolist=True,
    enable_filesystem=True,
    enable_subagents=True,
    enable_summarization=True,
    subagents=custom_subagents,
    max_tokens_before_summary=50_000,
    messages_to_keep=6,
)

print("✓ Full Deep Agent создан со всеми компонентами:")
print(f"  Middleware: {[type(m).__name__ for m in deep_agent.middlewares.middlewares]}")
print(f"  Tools: {[tool.name for tool in deep_agent.tool_executor.registry.list_tools()]}")
print(f"  Subagents: general-purpose, analyzer, writer")

In [ ]:
# Комплексная задача
complex_task = """
I need you to complete a comprehensive project analysis:

1. Create a project plan (use write_todos to break down the task)

2. Delegate the code analysis to the 'analyzer' subagent:
   - Analyze the middleware architecture
   - Identify key patterns
   - Suggest improvements

3. Create a report file 'analysis_report.md':
   - Title and introduction
   - Analysis findings from subagent
   - Recommendations section
   - Conclusion

4. Search the report for the word 'middleware'

5. Provide a final summary
"""

print("🎯 Комплексная задача:", complex_task.strip())
print("\n" + "="*80 + "\n")

async def run_complex_demo():
    result = ""
    async for chunk in deep_agent.run_stream(complex_task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_complex_demo()
print("\n" + "="*80)
print("\n✅ Комплексная задача выполнена!")

# Проверяем созданный файл
report_path = test_dir / "analysis_report.md"
if report_path.exists():
    print("\n📄 Создан файл: analysis_report.md")
    print(f"   Размер: {len(report_path.read_text())} символов")

## 8. Статистика и анализ

Проверяем статистику работы агента.

In [ ]:
# Анализируем state агента
messages = deep_agent.state.get("messages", [])
iteration = deep_agent.state.get("iteration", 0)
files = deep_agent.state.get("files", {})

print("📊 Статистика Deep Agent:")
print(f"  Сообщений в истории: {len(messages)}")
print(f"  Итераций выполнено: {iteration}")
print(f"  Файлов в virtual FS: {len(files)}")
print(f"  Токенов в контексте: {count_messages_tokens(messages)}")

# Считаем вызовы инструментов
tool_calls = {}
for msg in messages:
    if msg.get("role") == "assistant" and "tool_calls" in msg:
        for tc in msg["tool_calls"]:
            tool_name = tc["function"]["name"]
            tool_calls[tool_name] = tool_calls.get(tool_name, 0) + 1

print("\n🔧 Использование инструментов:")
for tool_name, count in sorted(tool_calls.items(), key=lambda x: x[1], reverse=True):
    print(f"  {tool_name}: {count}x")

# Анализируем типы сообщений
message_types = {}
for msg in messages:
    role = msg.get("role", "unknown")
    message_types[role] = message_types.get(role, 0) + 1

print("\n💬 Распределение сообщений:")
for role, count in sorted(message_types.items(), key=lambda x: x[1], reverse=True):
    print(f"  {role}: {count}")

## 9. Сравнение с Deep Agents (LangChain)

Наша реализация vs оригинальная.

In [ ]:
comparison = {
    "Component": [
        "TodoListMiddleware",
        "FilesystemMiddleware",
        "SubAgentMiddleware",
        "SummarizationMiddleware",
        "File Tools",
        "Execute Tool",
        "Prompt Caching",
    ],
    "Deep Agents (LangChain)": [
        "✓ Full",
        "✓ Full",
        "✓ Full",
        "✓ 170k tokens",
        "✓ 7 tools",
        "✓ With sandbox",
        "✓ Anthropic only",
    ],
    "Our Implementation": [
        "✓ Full",
        "✓ Context eviction",
        "✓ Full",
        "✓ 50k tokens",
        "✓ 6 tools",
        "✗ No sandbox",
        "✗ Provider-agnostic",
    ],
}

print("📋 Сравнение компонентов:\n")
print(f"{'Component':<30} {'Deep Agents':<25} {'Our Implementation':<25}")
print("="*80)

for i in range(len(comparison["Component"])):
    comp = comparison["Component"][i]
    da = comparison["Deep Agents (LangChain)"][i]
    ours = comparison["Our Implementation"][i]
    print(f"{comp:<30} {da:<25} {ours:<25}")

print("\n✅ Ключевые преимущества нашей реализации:")
print("  1. Независимость от LangChain/LangGraph")
print("  2. Поддержка любых LLM providers (не только Anthropic)")
print("  3. Enable/disable флаги для гибкого контроля")
print("  4. Fallback механизм в summarization")
print("  5. Простая архитектура без лишних зависимостей")

print("\n⚠️ Компромиссы:")
print("  1. Нет execute tool (требует sandbox backend)")
print("  2. 50k token threshold вместо 170k (для экономии)")
print("  3. Нет vendor-specific оптимизаций (prompt caching)")

## 10. Cleanup

In [ ]:
# Очищаем тестовые файлы
if test_dir.exists():
    shutil.rmtree(test_dir)
    print(f"✓ Удалён тестовый проект: {test_dir}")

print("\n✅ Demo завершено!")